In [1]:
import torch

In [12]:
out = torch.nn.functional.softmax(torch.rand((8, 100,)), dim=1)

In [18]:
clses = out.argmax(dim=1)

In [ ]:
print((clses == clses).sum() / 8)

In [1]:
import numpy as np
from pathlib import Path
from PIL import Image

import shutil

In [2]:
data_root = Path("/mnt/data/documents/code/ml-projects/ViT-TensorRT/datasets/dummy")
shutil.rmtree(data_root)

In [3]:
train_dir = data_root / "train"
valid_dir = data_root / "valid"

train_image_dir = train_dir / "images"
train_label_dir = train_dir / "labels"

valid_image_dir = valid_dir / "images"
valid_label_dir = valid_dir / "labels"

train_image_dir.mkdir(exist_ok=True, parents=True)
train_label_dir.mkdir(exist_ok=True, parents=True)

valid_image_dir.mkdir(exist_ok=True, parents=True)
valid_label_dir.mkdir(exist_ok=True, parents=True)

In [4]:
# Make the training dataset
for idx in range(100):
    if idx % 3 == 0:
        class_id = 0
        im = np.zeros((320, 320, 3), dtype=np.uint8)
    elif idx % 3 == 1:
        class_id = 1
        im = np.ones((320, 320, 3), dtype=np.uint8) * 127
    else:
        class_id = 2
        im = np.ones((320, 320, 3), dtype=np.uint8) * 255

    im = Image.fromarray(im)
    im.save(train_image_dir / f"{idx:04}.png")
    
    with open(train_label_dir / f"{idx:04}.txt", "w") as fp:
        fp.write(f"{class_id}")

In [5]:
# Make the validation dataset
for idx in range(100):
    if idx % 3 == 0:
        class_id = 0
        im = np.zeros((320, 320, 3), dtype=np.uint8)
    elif idx % 3 == 1:
        class_id = 1
        im = np.ones((320, 320, 3), dtype=np.uint8) * 127
    else:
        class_id = 2
        im = np.ones((320, 320, 3), dtype=np.uint8) * 255

    im = Image.fromarray(im)
    im.save(valid_image_dir / f"{idx:04}.png")
    
    with open(valid_label_dir / f"{idx:04}.txt", "w") as fp:
        fp.write(f"{class_id}")

In [7]:
from torch.utils.tensorboard import SummaryWriter

In [1]:
from vit_tensorrt import ViT
from vit_tensorrt.config import ViTConfig
from pathlib import Path
import torch

In [ ]:
model = ViT()

In [3]:
file = Path("/mnt/data/documents/code/ml-projects/ViT-TensorRT/model.pt")
model.save(file)

In [ ]:
loaded_model = ViT.load(file)

In [ ]:
loaded_model.config

### Cat dog classification

In [6]:
import random
from pathlib import Path

Pre-process the data into the format required for ViT training

In [7]:
data_dir = Path("/mnt/data/documents/code/ml-projects/ViT-TensorRT/datasets/cat-dog/train")
files = list(data_dir.glob("*.jpg"))

# Split into train and valid
random.seed(1234)
random.shuffle(files)
train_files = files[:20_000]
valid_files = files[20_000:]

# Save training data
train_dir = data_dir
train_images_dir = train_dir / "images"
train_labels_dir = train_dir / "labels"

train_images_dir.mkdir(exist_ok=True, parents=True)
train_labels_dir.mkdir(exist_ok=True, parents=True)

for file in train_files:
    with open(train_labels_dir / file.name, "w") as fp:
        if "dog" in file.stem:
            fp.write("1")
        elif "cat" in file.stem:
            fp.write("0")

    new_image_file = train_images_dir / file.name
    file.rename(new_image_file)


# Save validation data
valid_dir = data_dir.parent / "valid"
valid_images_dir = valid_dir / "images"
valid_labels_dir = valid_dir / "labels"

valid_images_dir.mkdir(exist_ok=True, parents=True)
valid_labels_dir.mkdir(exist_ok=True, parents=True)

for file in valid_files:
    with open(valid_labels_dir / file.name, "w") as fp:
        if "dog" in file.stem:
            fp.write("1")
        elif "cat" in file.stem:
            fp.write("0")

    new_image_file = valid_images_dir / file.name
    file.rename(new_image_file)

Train the model

In [1]:
from pathlib import Path

from vit_tensorrt import ViT
from vit_tensorrt.config import EncoderConfig, TrainConfig, ViTConfig

In [ ]:
encoder_config = EncoderConfig(num_layers=8)
config = ViTConfig(image_size=320, num_classes=2, encoder_config=encoder_config)
model = ViT(config, device="cuda:0")

In [ ]:
train_config = TrainConfig(
    data_path=Path("/mnt/data/documents/code/ml-projects/ViT-TensorRT/datasets/cat-dog"),
    batch_size=128,
    learning_rate=1e-4
    )
model.fit(train_config)

### Investigate the data loader

In [1]:
from pathlib import Path

from vit_tensorrt import ViT
from vit_tensorrt.config import EncoderConfig, TrainConfig, ViTConfig

In [ ]:
encoder_config = EncoderConfig(num_layers=8)
config = ViTConfig(image_size=320, num_classes=2, encoder_config=encoder_config)
model = ViT(config, device="cuda:0")

train_config = TrainConfig(
    data_path=Path("/mnt/data/documents/code/ml-projects/ViT-TensorRT/datasets/cat-dog"),
    batch_size=128
    )

In [ ]:
train_loader, valid_loader = model._prepare_data(train_config)

In [4]:
images: torch.Tensor = None
labels: torch.Tensor = None
for images, labels in train_loader:
    break

In [ ]:
import matplotlib.pyplot as plt
index = 10

im = images[index].permute(1, 2, 0)
cpu_im = im.cpu().numpy()
print(labels[index])
print(cpu_im.max(), cpu_im.min())
plt.imshow(cpu_im)

In [1]:
import torch

t1 = torch.zeros((3, 10))
t2 = torch.randint(0, 10, (1, 10))

In [ ]:
print(t1)
print(t2)

In [ ]:
t1+t2

### Learning rate scheduler

In [1]:
from torch.optim.lr_scheduler import ChainedScheduler, CosineAnnealingLR, LinearLR
from torch.optim import Adam

In [ ]:
encoder_config = EncoderConfig(num_layers=8)
config = ViTConfig(image_size=320, num_classes=2, encoder_config=encoder_config)
model = ViT(config, device="cuda:0")

optimiser = Adam(model.parameters(), lr=1e-4)

scheduler = ChainedScheduler([LinearLR(optimiser, last_epoch=5)], optimiser)